In [73]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/credit-card-fraud/card_transdata.csv
/kaggle/input/card-transdata-test/card_transdata_test.csv
/kaggle/input/fraud1test/card_transdata_test_fraud.csv


In [74]:
from sklearn.model_selection import train_test_split

# Read the data
#The source of dataset - https://www.kaggle.com/datasets/dhanushnarayananr/credit-card-fraud
X = pd.read_csv('../input/credit-card-fraud/card_transdata.csv')
X_test_full = pd.read_csv('../input/card-transdata-test/card_transdata_test.csv')

print('X and X_test before data preparation:')
print(X.shape)
print(X_test_full.shape)

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['fraud'], inplace=True)
y = X.fraud              
X.drop(['fraud'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

###For large datasets with many rows, one-hot encoding can greatly expand the size of the dataset.  
###For this reason, we typically will only one-hot encode columns with relatively low cardinality.  
###Then, high cardinality columns can either be dropped from the dataset, or we can use ordinal encoding.

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

print()
print('X and X_test after data preparation:')
print(X_train.shape)
print(y.shape)

X and X_test before data preparation:
(1000000, 8)
(1156, 8)

X and X_test before data preparation:
(800000, 7)
(1000000,)


In [75]:
#Create pipeline:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numeric_cols),
        ('cat', categorical_transformer, low_cardinality_cols)
    ])

# Define model
#model = RandomForestRegressor(n_estimators=100, random_state=0)
#model = XGBRegressor(random_state=0)
model = XGBRegressor(n_estimators=1000, learning_rate=0.05)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))


MAE: 9.777023159497133e-05


In [ ]:
print(clf)

In [83]:
output = pd.DataFrame({'fraud': preds_test})
output.to_csv('submission1.csv', index=False)

In [104]:
x1 = pd.read_csv('../input/inputfraud1test/card_transdata_test_fraud1.csv')
preds1 = clf.predict(x1)
print(f'Prediction of one real transaction: {preds1}')

Prediction of one real transaction: [1.0013679]


In [101]:
#Try and compare different models

estimators = {
    'XGBRegressor1': XGBRegressor(n_estimators=200, learning_rate=0.05),
    'XGBRegressor2': XGBRegressor(n_estimators=100, learning_rate=1),
    'XGBRegressor3': XGBRegressor(random_state=0),
    'RandomForestRegressor': RandomForestRegressor(n_estimators=100, random_state=0),
    'XGBRegressor3': XGBRegressor(n_estimators=1000, learning_rate=0.05)
}

In [ ]:
from sklearn.model_selection import cross_val_score

for estimator_name, estimator_object in estimators.items():
    # Bundle preprocessing and modeling code in a pipeline
    clf = Pipeline(steps=[('preprocessor', preprocessor), ('model', estimator_object)])
    # Preprocessing of training data, fit model 
    clf.fit(X_train, y_train)
    # Preprocessing of validation data, get predictions
    preds = clf.predict(X_valid)
    
    print('MAE scoring')
    MAE = mean_absolute_error(y_valid, preds)
    print(f'{estimator_name:>16}: MAE: {MAE:.6f}')

    print('Cross_val_score')
    scores = -1 * cross_val_score(clf, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')
    print("Average MAE score (CV):", scores.mean())
    print()